##### Note: The summaries for the other layers will come soon in the same format after the clean up is done. 

# Summary of best models for given numbers of layers

In [ ]:
# Each row is the output for a model in the following format. Different lines for different model structures. See the cell below for the excecuted codes.
# print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {tloss:.4f}, {torch.Tensor(loss).mean():.4f}, {N_step}],')
# Example: [16layers, 12(6)heads, hidden-dim=696, inter-layer-dim = 696x4, head-dim=224, #param=218million, train_loss, val_loss, Num-steps]

[+36, 4, 2, 648, 2592, 160, 259.0, 2.4992,+2.4858, 144000], # minGemma-hidden_layers36-att_heads4-kv_heads2-hidden648-intermediate2592-head_dim160-T512--2025-07-02-14-17.pth
[+30, 4, 2, 648, 2592, 224, 236.2, 2.5071,+2.4812, 144000], # minGemma-hidden_layers30-att_heads4-kv_heads2-hidden648-intermediate2592-head_dim224-T512--2025-06-18-19-30.pth
[+24, 6, 3, 648, 2592, 272, 229.8, 2.5135,+2.4748, 144000], # minGemma-hidden_layers24-att_heads6-kv_heads3-hidden648-intermediate2592-head_dim272-T512--2025-07-13-19-45.pth

# L36 (Models with 36 Layers)

In [ ]:
# L36 Normal Model (default: x4 B28 lr10e-4 WD0.25 N144000)
[ 36, 5, 5, 640, 3840, 144, 364.3, 2.4763, 2.5109, 144000], # 41.2h 42.6G x6
[ 36, 5, 5, 640, 2560, 144, 275.7, 2.4952, 2.4890, 144000], # 36.5h 34.9G x4
[ 36, 4, 4, 600, 2400, 144, 235.7, 2.5155, 2.4896, 144000], # 32.8h 33.8G x4

[ 36, 4, 2, 696, 3480, 224, 364.2, 2.5042, 2.5060, 144000], # 43.4h 41.9G x5 TOO big # best normal model
[ 36, 4, 2, 696, 2784, 224, 311.9, 2.4954, 2.4920, 144000], # 40.4h 39.0G x4 TOO big
[ 36, 4, 2, 696, 2784, 192, 302.3, 2.4950, 2.5004, 144000], # 38.3h       x4 TOO big
[ 36, 4, 2, 696, 2784, 160, 292.6, 2.5039, 2.4934, 144000], # 36.5h       x4 TOO big
[ 36, 4, 2, 648, 2592, 224, 276.9, 2.4947, 2.4918, 144000], # 38.6h 35.1G x4 TOO big
[ 36, 4, 2, 648, 2592, 192, 268.0, 2.4897, 2.4893, 144000], # 36.8h       x4 TOO big
[+36, 4, 2, 648, 2592, 160, 259.0, 2.4992,+2.4858, 144000], # 35.0h       x4 (MIN confirmed)  minGemma-hidden_layers36-att_heads4-kv_heads2-hidden648-intermediate2592-head_dim160-T512--2025-07-02-14-17
[ 36, 4, 2, 648, 2592, 144, 254.5, 2.4979, 2.4899, 144000], # 34.3h       x4 TOO small
[ 36, 4, 2, 600, 2400, 160, 227.4, 2.5029, 2.4865, 144000], # 32.6h       x4 TOO small
[ 36, 4, 2, 600, 2400, 144, 223.2, 2.5039, 2.4899, 144000], # 32.0h       x4 TOO small

In [8]:
import matplotlib.pyplot as plt; import numpy as np; import time, torch; device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoTokenizer, TrainingArguments, DefaultDataCollator, Trainer
vocab_size = 50257 # =tokenizer.vocab_size  # FIX!!! # G256128    ### T=256 for minGemma # G8192 for real Gemma
num_hidden_layers =  36 # 8 # G28 G18 #blocks
num_attention_heads = 4 # 4 # G16 G8
num_key_value_heads = 2 # 4 # G16 G1
hidden_size = num_attention_heads*162 # 128 # G3072 G2048 # embedding dimension
intermediate_size = hidden_size*4 # x4 or x8 # time limiting factor #512 # G24576 G16384  # MLP inner dim
head_dim = 160 # 32 # G256 # dim in attention # Doesn't affect time
rms_norm_eps = 1e-4 # 1e-6
rope_theta = 4000.0 # scale freq is small for S-model. 1000 might work too # G10000.0

def apply_rotary_emb(x: torch.Tensor, dim: int) -> torch.Tensor: # seq_len = x.size(1) # N
    freqs = 1.0 / (rope_theta ** (torch.arange(0, dim, 2, device=device).float() / dim)) # Dynamically compute frequency cis
    t = torch.arange(x.size(1), device=device); freqs = torch.outer(t, freqs).float(); freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis.unsqueeze(0)).type_as(x)  # Ensure batch dimension is handled
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    return x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)

class RMSNorm(torch.nn.Module): # RMS:4.326552, RMS_no_weight:4.410741 # RMS':4.554899
    def __init__(self, dim: int = hidden_size):
        super().__init__(); self.weight = torch.nn.Parameter(torch.zeros(dim)) # one weight per feature to be learned
    def _norm(self, x): # mean square for each feature (across the last dimension)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + rms_norm_eps)
    def forward(self, x): # ensure the data type matches the input.
        return self._norm(x.float()).type_as(x) * (1 + self.weight)

class GemmaAttention(torch.nn.Module): # MQA = K,V shared by 4Qs
    def __init__(self):
        super().__init__(); self.qkv_proj = torch.nn.Linear(hidden_size, (num_attention_heads + 2 * num_key_value_heads) * head_dim, bias=False); self.o_proj = torch.nn.Linear(num_attention_heads * head_dim, hidden_size, bias=False) # concatenated attention outputs back to the hidden size.
    def forward(self, hidden_states: torch.Tensor,) -> torch.Tensor:  # in=(B, T, hidden_size)
        batch_size, input_len, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states)
        xq, xk, xv = qkv.split([num_attention_heads * head_dim, num_key_value_heads * head_dim, num_key_value_heads * head_dim],dim=-1)
        xq = xq.view(batch_size, -1, num_attention_heads, head_dim); xk = xk.view(batch_size, -1, num_key_value_heads, head_dim); xv = xv.view(batch_size, -1, num_key_value_heads, head_dim)
        xq = apply_rotary_emb(xq, head_dim); xk = apply_rotary_emb(xk, head_dim)
        if num_key_value_heads != num_attention_heads:  # Q/KV multiples of K and V to match Q
            xk = torch.repeat_interleave(xk, num_attention_heads // num_key_value_heads, dim=2) # [B, T, n_local_heads, head_dim]
            xv = torch.repeat_interleave(xv, num_attention_heads // num_key_value_heads, dim=2)
        q = xq.transpose(1, 2); k = xk.transpose(1, 2); v = xv.transpose(1, 2) # [batch_size, n_local_heads, input_len, head_dim]
        output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs        
        output = output.transpose(1, 2).contiguous().view(batch_size, input_len, -1)  # [B, T, "hidden_dim"]
        return self.o_proj(output)

class GemmaDecoderLayer(torch.nn.Module): # normalize before and after the attention mechanism
    def __init__(self):
        super().__init__(); self.self_attn = GemmaAttention(); self.input_layernorm = RMSNorm(); self.post_attention_layernorm = RMSNorm(); self.gate_proj = torch.nn.Linear(hidden_size, intermediate_size); self.up_proj = torch.nn.Linear(hidden_size, intermediate_size); self.down_proj = torch.nn.Linear(intermediate_size, hidden_size) # mlp
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:  # input_size = (B, T, hidden_size)
        residual = hidden_states # Self Attention Block
        hidden_states = self.input_layernorm(hidden_states); hidden_states = self.self_attn(hidden_states=hidden_states)
        hidden_states = residual + hidden_states
        residual = hidden_states # MLP Block
        hidden_states = self.post_attention_layernorm(hidden_states); gate = torch.nn.functional.gelu(self.gate_proj(hidden_states)); up = self.up_proj(hidden_states); fuse = gate * up; hidden_states = self.down_proj(fuse) # mlp
        return residual + hidden_states

class minGemma(torch.nn.Module):
    def __init__(self):
        super().__init__(); self.embedder = torch.nn.Embedding(vocab_size, hidden_size); self.layers = torch.nn.ModuleList(GemmaDecoderLayer() for _ in range(num_hidden_layers)); self.norm = RMSNorm();
    def forward(self, input_token_ids: torch.Tensor) -> torch.Tensor: # (B, T)
        hidden_states = self.embedder(input_token_ids[:,:-1]) # (B, T) & (vocab_size, hidden_size) -> (B, T, hidden_size)
        hidden_states = hidden_states * (hidden_size**0.5)
        for i in range(len(self.layers)):
            hidden_states = self.layers[i](hidden_states) # shortened too much???
        hidden_states = self.norm(hidden_states) # -> (B, T, hidden_size)        
        embedder_weight = self.embedder.weight
        logits = torch.matmul(hidden_states, embedder_weight.t()); b,t,v=logits.shape; # (B, T, hidden_size) @ (hidden_size, vocab_size) -> (B, T, vocab_size)
        loss = torch.nn.functional.cross_entropy(logits.view(b*t,v), input_token_ids[:,1:].reshape(b*t)) #, weight=None, ignore_index=-100, reduction='mean')
        return loss, logits # logits, loss

def map_to_array5(ix):
    common = torch.stack([torch.from_numpy((train_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}
def map_to_array_Val(ix):
    common = torch.stack([torch.from_numpy((val_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}

train_data = np.memmap('train_BabyLM_100M.bin', dtype=np.uint16, mode='r'); val_data = np.memmap('val_BabyLM.bin', dtype=np.uint16, mode='r')
T=512; B=28; N_step=144000; print(T * B * N_step / 1000000) # 0.25 B-tokens being calculated # n_steps=N_step;
model = minGemma().to(device); print(f'L{num_hidden_layers}' f' att{num_attention_heads}' f' kv_heads{num_key_value_heads}' f' hidden{hidden_size}' f' intermediate{intermediate_size}' f' head_dim{head_dim}' f' T{T}')
torch.cuda.empty_cache();

# Normal Model with Hand-eval
training_args = TrainingArguments(learning_rate=10e-4, weight_decay=0.25, num_train_epochs=1, logging_strategy='epoch', output_dir='./test', bf16=True, per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='no', save_strategy='no', report_to='none', remove_unused_columns=False, dataloader_pin_memory=True) #, dataloader_num_workers=4
trainer = Trainer(model=model, args=training_args, train_dataset=torch.utils.data.TensorDataset(torch.randint(len(train_data)-T-1, (B*N_step,))), data_collator=map_to_array5);
result = trainer.train(); tloss=result[2]["train_loss"]
loss = []; model.eval(); B2=16; torch.cuda.empty_cache();
for k in range(4000):
    val_ind = torch.randint(len(val_data)-T-1, (B2,)); common = (torch.stack([torch.from_numpy((val_data[i:i+T+1]).astype(np.int64)) for i in val_ind]))
    loss += [model(common.to('cuda', non_blocking=True))[0].item()]
if torch.Tensor(loss).mean() < 2.4858:
    torch.save(model.state_dict(), f'{model.__class__.__name__}' f'-hidden_layers{num_hidden_layers}' f'-att_heads{num_attention_heads}' f'-kv_heads{num_key_value_heads}' f'-hidden{hidden_size}' f'-intermediate{intermediate_size}' f'-head_dim{head_dim}' f'-T{T}' f'--{time.strftime("%Y-%m-%d-%H-%M")}.pth')
model.train(); del common; print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {tloss:.4f}, {torch.Tensor(loss).mean():.4f}, {N_step}],')

2064.384
L36 att4 kv_heads2 hidden648 intermediate2592 head_dim160 T512


Step,Training Loss
144000,2.499200


[ 36, 4, 2, 648, 2592, 160, 259.0, 2.4992, 2.4858, 144000],


# L30

In [ ]:
# L30 Normal Model (default: x4 B28 lr10e-4 WD0.25 N144000)

[ 30, 6, 3, 720, 2880, 160, 285.3, 2.4815, 2.4926, 144000], # 36.5h 36.3G
[ 30, 5, 5, 640, 2560, 144, 235.1, 2.5239, 2.4838, 144000], # 31.1h 33.2G (if x6 37.8G)
[ 30, 5, 1, 720, 2880, 288, 297.7, 2.4904, 2.4944, 144000], # 41.2h 34.7G # 2nd best

[ 30, 4, 2, 696, 2784, 224, 265.7, 2.5058, 2.4898, 144000], # 34.3h       x4 TOO big
[ 30, 4, 2, 648, 2592, 256, 243.7, 2.4909, 2.4829, 144000], # 34.2h       x4 TOO big
[+30, 4, 2, 648, 2592, 224, 236.2, 2.5071,+2.4812, 144000], # 32.7h 31.3G x4 (MIN confirmed) # best 10M normal # minGemma-hidden_layers30-att_heads4-kv_heads2-hidden648-intermediate2592-head_dim224-T512--2025-06-18-19-30.pth
[ 30, 4, 2, 648, 2592, 192, 228.7, 2.5032, 2.4832, 144000], # 31.3h 30.9G x4 TOO small
[ 30, 4, 2, 600, 2400, 224, 208.3, 2.5080, 2.4820, 144000], # 30.9h 29.8G x4 TOO small

In [14]:
import matplotlib.pyplot as plt; import numpy as np; import time, torch; device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoTokenizer, TrainingArguments, DefaultDataCollator, Trainer
vocab_size = 50257 # =tokenizer.vocab_size  # FIX!!! # G256128    ### T=256 for minGemma # G8192 for real Gemma
num_hidden_layers =  30 # 8 # G28 G18 #blocks
num_attention_heads = 4 # 4 # G16 G8
num_key_value_heads = 2 # 4 # G16 G1
hidden_size = num_attention_heads*150 # 128 # G3072 G2048 # embedding dimension
intermediate_size = hidden_size*4 # x4 or x8 # time limiting factor #512 # G24576 G16384  # MLP inner dim
head_dim = 224 # 32 # G256 # dim in attention # Doesn't affect time
rms_norm_eps = 1e-4 # 1e-6
rope_theta = 4000.0 # scale freq is small for S-model. 1000 might work too # G10000.0

def apply_rotary_emb(x: torch.Tensor, dim: int) -> torch.Tensor: # seq_len = x.size(1) # N
    freqs = 1.0 / (rope_theta ** (torch.arange(0, dim, 2, device=device).float() / dim)) # Dynamically compute frequency cis
    t = torch.arange(x.size(1), device=device); freqs = torch.outer(t, freqs).float(); freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis.unsqueeze(0)).type_as(x)  # Ensure batch dimension is handled
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    return x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)

class RMSNorm(torch.nn.Module): # RMS:4.326552, RMS_no_weight:4.410741 # RMS':4.554899
    def __init__(self, dim: int = hidden_size):
        super().__init__(); self.weight = torch.nn.Parameter(torch.zeros(dim)) # one weight per feature to be learned
    def _norm(self, x): # mean square for each feature (across the last dimension)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + rms_norm_eps)
    def forward(self, x): # ensure the data type matches the input.
        return self._norm(x.float()).type_as(x) * (1 + self.weight)

class GemmaAttention(torch.nn.Module): # MQA = K,V shared by 4Qs
    def __init__(self):
        super().__init__(); self.qkv_proj = torch.nn.Linear(hidden_size, (num_attention_heads + 2 * num_key_value_heads) * head_dim, bias=False); self.o_proj = torch.nn.Linear(num_attention_heads * head_dim, hidden_size, bias=False) # concatenated attention outputs back to the hidden size.
    def forward(self, hidden_states: torch.Tensor,) -> torch.Tensor:  # in=(B, T, hidden_size)
        batch_size, input_len, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states)
        xq, xk, xv = qkv.split([num_attention_heads * head_dim, num_key_value_heads * head_dim, num_key_value_heads * head_dim],dim=-1)
        xq = xq.view(batch_size, -1, num_attention_heads, head_dim); xk = xk.view(batch_size, -1, num_key_value_heads, head_dim); xv = xv.view(batch_size, -1, num_key_value_heads, head_dim)
        xq = apply_rotary_emb(xq, head_dim); xk = apply_rotary_emb(xk, head_dim)
        if num_key_value_heads != num_attention_heads:  # Q/KV multiples of K and V to match Q
            xk = torch.repeat_interleave(xk, num_attention_heads // num_key_value_heads, dim=2) # [B, T, n_local_heads, head_dim]
            xv = torch.repeat_interleave(xv, num_attention_heads // num_key_value_heads, dim=2)
        q = xq.transpose(1, 2); k = xk.transpose(1, 2); v = xv.transpose(1, 2) # [batch_size, n_local_heads, input_len, head_dim]
        output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs        
        output = output.transpose(1, 2).contiguous().view(batch_size, input_len, -1)  # [B, T, "hidden_dim"]
        return self.o_proj(output)

class GemmaDecoderLayer(torch.nn.Module): # normalize before and after the attention mechanism
    def __init__(self):
        super().__init__(); self.self_attn = GemmaAttention(); self.input_layernorm = RMSNorm(); self.post_attention_layernorm = RMSNorm(); self.gate_proj = torch.nn.Linear(hidden_size, intermediate_size); self.up_proj = torch.nn.Linear(hidden_size, intermediate_size); self.down_proj = torch.nn.Linear(intermediate_size, hidden_size) # mlp
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:  # input_size = (B, T, hidden_size)
        residual = hidden_states # Self Attention Block
        hidden_states = self.input_layernorm(hidden_states); hidden_states = self.self_attn(hidden_states=hidden_states)
        hidden_states = residual + hidden_states
        residual = hidden_states # MLP Block
        hidden_states = self.post_attention_layernorm(hidden_states); gate = torch.nn.functional.gelu(self.gate_proj(hidden_states)); up = self.up_proj(hidden_states); fuse = gate * up; hidden_states = self.down_proj(fuse) # mlp
        return residual + hidden_states

class minGemma(torch.nn.Module):
    def __init__(self):
        super().__init__(); self.embedder = torch.nn.Embedding(vocab_size, hidden_size); self.layers = torch.nn.ModuleList(GemmaDecoderLayer() for _ in range(num_hidden_layers)); self.norm = RMSNorm();
    def forward(self, input_token_ids: torch.Tensor) -> torch.Tensor: # (B, T)
        hidden_states = self.embedder(input_token_ids[:,:-1]) # (B, T) & (vocab_size, hidden_size) -> (B, T, hidden_size)
        hidden_states = hidden_states * (hidden_size**0.5)
        for i in range(len(self.layers)):
            hidden_states = self.layers[i](hidden_states) # shortened too much???
        hidden_states = self.norm(hidden_states) # -> (B, T, hidden_size)        
        embedder_weight = self.embedder.weight
        logits = torch.matmul(hidden_states, embedder_weight.t()); b,t,v=logits.shape; # (B, T, hidden_size) @ (hidden_size, vocab_size) -> (B, T, vocab_size)
        loss = torch.nn.functional.cross_entropy(logits.view(b*t,v), input_token_ids[:,1:].reshape(b*t)) #, weight=None, ignore_index=-100, reduction='mean')
        return loss, logits # logits, loss

def map_to_array5(ix):
    common = torch.stack([torch.from_numpy((train_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}
def map_to_array_Val(ix):
    common = torch.stack([torch.from_numpy((val_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}

train_data = np.memmap('train_BabyLM_100M.bin', dtype=np.uint16, mode='r'); val_data = np.memmap('val_BabyLM.bin', dtype=np.uint16, mode='r')
T=512; B=28; N_step=144000; print(T * B * N_step / 1000000) # 0.25 B-tokens being calculated # n_steps=N_step;
model = minGemma().to(device); print(f'L{num_hidden_layers}' f' att{num_attention_heads}' f' kv_heads{num_key_value_heads}' f' hidden{hidden_size}' f' intermediate{intermediate_size}' f' head_dim{head_dim}' f' T{T}')
torch.cuda.empty_cache();

# Normal Model with Hand-eval
training_args = TrainingArguments(learning_rate=10e-4, weight_decay=0.25, num_train_epochs=1, logging_strategy='epoch', output_dir='./test', bf16=True, per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='no', save_strategy='no', report_to='none', remove_unused_columns=False, dataloader_pin_memory=True) #, dataloader_num_workers=4
trainer = Trainer(model=model, args=training_args, train_dataset=torch.utils.data.TensorDataset(torch.randint(len(train_data)-T-1, (B*N_step,))), data_collator=map_to_array5);
result = trainer.train(); tloss=result[2]["train_loss"]
loss = []; model.eval(); B2=16; torch.cuda.empty_cache();
for k in range(4000):
    val_ind = torch.randint(len(val_data)-T-1, (B2,)); common = (torch.stack([torch.from_numpy((val_data[i:i+T+1]).astype(np.int64)) for i in val_ind]))
    loss += [model(common.to('cuda', non_blocking=True))[0].item()]
if torch.Tensor(loss).mean() < 2.4812:
    torch.save(model.state_dict(), f'{model.__class__.__name__}' f'-hidden_layers{num_hidden_layers}' f'-att_heads{num_attention_heads}' f'-kv_heads{num_key_value_heads}' f'-hidden{hidden_size}' f'-intermediate{intermediate_size}' f'-head_dim{head_dim}' f'-T{T}' f'--{time.strftime("%Y-%m-%d-%H-%M")}.pth')
model.train(); del common; print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {tloss:.4f}, {torch.Tensor(loss).mean():.4f}, {N_step}],')

2064.384
L30 att4 kv_heads2 hidden600 intermediate2400 head_dim224 T512


Step,Training Loss
144000,2.508000


[ 30, 4, 2, 600, 2400, 224, 208.3, 2.5080, 2.4820, 144000],


# L24

In [ ]:
# L24 Normal Model (default: x4 B28 lr10e-4 WD0.25 N144000)
[ 24, 8, 4, 704, 2816, 144, 236.7, 2.4969, 2.4841, 144000], # 32.0h 29.1G x4
[ 24, 6, 3, 696, 2784, 240, 246.8, 2.5042, 2.4846, 144000], # 34.4h 29.8G x4 TOO big
[ 24, 6, 3, 696, 2784, 272, 256.5, 2.4990, 2.4836, 144000], # 37.5h 33.1G x4 TOO big
[ 24, 6, 3, 648, 2592, 304, 238.8, 2.5055, 2.4799, 144000], # 38.3h 30.1G x4 TOO big
[+24, 6, 3, 648, 2592, 272, 229.8, 2.5135,+2.4748, 144000], # 36.5h 29.2G x4 (MIN confirmed) minGemma-hidden_layers24-att_heads6-kv_heads3-hidden648-intermediate2592-head_dim272-T512--2025-07-13-19-45.pth
[ 24, 6, 3, 648, 3888, 240, 281.4, 2.4856, 2.4908, 144000], # 36.4h 34.6G x6 TOO big
[ 24, 6, 3, 648, 3240, 240, 251.1, 2.5044, 2.4878, 144000], # 35.1h 30.3G x5 TOO big
[ 24, 6, 3, 648, 2592, 240, 220.9, 2.5138,+2.4805, 144000], # 33.2h       x4 TOO small # best normal
[ 24, 6, 3, 600, 2400, 272, 204.5, 2.5241, 2.4817, 144000], # 34.7h 28.6G x4 TOO small

In [16]:
import matplotlib.pyplot as plt; import numpy as np; import time, torch; device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoTokenizer, TrainingArguments, DefaultDataCollator, Trainer
vocab_size = 50257 # =tokenizer.vocab_size  # FIX!!! # G256128    ### T=256 for minGemma # G8192 for real Gemma
num_hidden_layers =  24 # 8 # G28 G18 #blocks
num_attention_heads = 6 # 4 # G16 G8
num_key_value_heads = 3 # 4 # G16 G1
hidden_size = num_attention_heads*108 # 128 # G3072 G2048 # embedding dimension
intermediate_size = hidden_size*4 # x4 or x8 # time limiting factor #512 # G24576 G16384  # MLP inner dim
head_dim = 272 # 32 # G256 # dim in attention # Doesn't affect time
rms_norm_eps = 1e-4 # 1e-6
rope_theta = 4000.0 # scale freq is small for S-model. 1000 might work too # G10000.0

def apply_rotary_emb(x: torch.Tensor, dim: int) -> torch.Tensor: # seq_len = x.size(1) # N
    freqs = 1.0 / (rope_theta ** (torch.arange(0, dim, 2, device=device).float() / dim)) # Dynamically compute frequency cis
    t = torch.arange(x.size(1), device=device); freqs = torch.outer(t, freqs).float(); freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    x_ = torch.view_as_complex(torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1), dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis.unsqueeze(0)).type_as(x)  # Ensure batch dimension is handled
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    return x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2], -1).transpose(1, 2)

class RMSNorm(torch.nn.Module): # RMS:4.326552, RMS_no_weight:4.410741 # RMS':4.554899
    def __init__(self, dim: int = hidden_size):
        super().__init__(); self.weight = torch.nn.Parameter(torch.zeros(dim)) # one weight per feature to be learned
    def _norm(self, x): # mean square for each feature (across the last dimension)
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + rms_norm_eps)
    def forward(self, x): # ensure the data type matches the input.
        return self._norm(x.float()).type_as(x) * (1 + self.weight)

class GemmaAttention(torch.nn.Module): # MQA = K,V shared by 4Qs
    def __init__(self):
        super().__init__(); self.qkv_proj = torch.nn.Linear(hidden_size, (num_attention_heads + 2 * num_key_value_heads) * head_dim, bias=False); self.o_proj = torch.nn.Linear(num_attention_heads * head_dim, hidden_size, bias=False) # concatenated attention outputs back to the hidden size.
    def forward(self, hidden_states: torch.Tensor,) -> torch.Tensor:  # in=(B, T, hidden_size)
        batch_size, input_len, _ = hidden_states.shape
        qkv = self.qkv_proj(hidden_states)
        xq, xk, xv = qkv.split([num_attention_heads * head_dim, num_key_value_heads * head_dim, num_key_value_heads * head_dim],dim=-1)
        xq = xq.view(batch_size, -1, num_attention_heads, head_dim); xk = xk.view(batch_size, -1, num_key_value_heads, head_dim); xv = xv.view(batch_size, -1, num_key_value_heads, head_dim)
        xq = apply_rotary_emb(xq, head_dim); xk = apply_rotary_emb(xk, head_dim)
        if num_key_value_heads != num_attention_heads:  # Q/KV multiples of K and V to match Q
            xk = torch.repeat_interleave(xk, num_attention_heads // num_key_value_heads, dim=2) # [B, T, n_local_heads, head_dim]
            xv = torch.repeat_interleave(xv, num_attention_heads // num_key_value_heads, dim=2)
        q = xq.transpose(1, 2); k = xk.transpose(1, 2); v = xv.transpose(1, 2) # [batch_size, n_local_heads, input_len, head_dim]
        output = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0, is_causal=True) # B nh T hs        
        output = output.transpose(1, 2).contiguous().view(batch_size, input_len, -1)  # [B, T, "hidden_dim"]
        return self.o_proj(output)

class GemmaDecoderLayer(torch.nn.Module): # normalize before and after the attention mechanism
    def __init__(self):
        super().__init__(); self.self_attn = GemmaAttention(); self.input_layernorm = RMSNorm(); self.post_attention_layernorm = RMSNorm(); self.gate_proj = torch.nn.Linear(hidden_size, intermediate_size); self.up_proj = torch.nn.Linear(hidden_size, intermediate_size); self.down_proj = torch.nn.Linear(intermediate_size, hidden_size) # mlp
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:  # input_size = (B, T, hidden_size)
        residual = hidden_states # Self Attention Block
        hidden_states = self.input_layernorm(hidden_states); hidden_states = self.self_attn(hidden_states=hidden_states)
        hidden_states = residual + hidden_states
        residual = hidden_states # MLP Block
        hidden_states = self.post_attention_layernorm(hidden_states); gate = torch.nn.functional.gelu(self.gate_proj(hidden_states)); up = self.up_proj(hidden_states); fuse = gate * up; hidden_states = self.down_proj(fuse) # mlp
        return residual + hidden_states

class minGemma(torch.nn.Module):
    def __init__(self):
        super().__init__(); self.embedder = torch.nn.Embedding(vocab_size, hidden_size); self.layers = torch.nn.ModuleList(GemmaDecoderLayer() for _ in range(num_hidden_layers)); self.norm = RMSNorm();
    def forward(self, input_token_ids: torch.Tensor) -> torch.Tensor: # (B, T)
        hidden_states = self.embedder(input_token_ids[:,:-1]) # (B, T) & (vocab_size, hidden_size) -> (B, T, hidden_size)
        hidden_states = hidden_states * (hidden_size**0.5)
        for i in range(len(self.layers)):
            hidden_states = self.layers[i](hidden_states) # shortened too much???
        hidden_states = self.norm(hidden_states) # -> (B, T, hidden_size)        
        embedder_weight = self.embedder.weight
        logits = torch.matmul(hidden_states, embedder_weight.t()); b,t,v=logits.shape; # (B, T, hidden_size) @ (hidden_size, vocab_size) -> (B, T, vocab_size)
        loss = torch.nn.functional.cross_entropy(logits.view(b*t,v), input_token_ids[:,1:].reshape(b*t)) #, weight=None, ignore_index=-100, reduction='mean')
        return loss, logits # logits, loss

def map_to_array5(ix):
    common = torch.stack([torch.from_numpy((train_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}
def map_to_array_Val(ix):
    common = torch.stack([torch.from_numpy((val_data[i[0]:i[0]+T+1]).astype(np.int64)) for i in ix]); return {'input_token_ids': common}

train_data = np.memmap('train_BabyLM_100M.bin', dtype=np.uint16, mode='r'); val_data = np.memmap('val_BabyLM.bin', dtype=np.uint16, mode='r')
T=512; B=28; N_step=144000; print(T * B * N_step / 1000000) # 0.25 B-tokens being calculated # n_steps=N_step;
model = minGemma().to(device); print(f'L{num_hidden_layers}' f' att{num_attention_heads}' f' kv_heads{num_key_value_heads}' f' hidden{hidden_size}' f' intermediate{intermediate_size}' f' head_dim{head_dim}' f' T{T}')
torch.cuda.empty_cache();

# Normal Model with Hand-eval
training_args = TrainingArguments(learning_rate=10e-4, weight_decay=0.25, num_train_epochs=1, logging_strategy='epoch', output_dir='./test', bf16=True, per_device_train_batch_size=B, per_device_eval_batch_size=B, eval_strategy='no', save_strategy='no', report_to='none', remove_unused_columns=False, dataloader_pin_memory=True) #, dataloader_num_workers=4
trainer = Trainer(model=model, args=training_args, train_dataset=torch.utils.data.TensorDataset(torch.randint(len(train_data)-T-1, (B*N_step,))), data_collator=map_to_array5);
result = trainer.train(); tloss=result[2]["train_loss"]
loss = []; model.eval(); B2=16; torch.cuda.empty_cache();
for k in range(4000):
    val_ind = torch.randint(len(val_data)-T-1, (B2,)); common = (torch.stack([torch.from_numpy((val_data[i:i+T+1]).astype(np.int64)) for i in val_ind]))
    loss += [model(common.to('cuda', non_blocking=True))[0].item()]
if torch.Tensor(loss).mean() < 2.4805:
    torch.save(model.state_dict(), f'{model.__class__.__name__}' f'-hidden_layers{num_hidden_layers}' f'-att_heads{num_attention_heads}' f'-kv_heads{num_key_value_heads}' f'-hidden{hidden_size}' f'-intermediate{intermediate_size}' f'-head_dim{head_dim}' f'-T{T}' f'--{time.strftime("%Y-%m-%d-%H-%M")}.pth')
model.train(); del common; print(f'[ {num_hidden_layers}, {num_attention_heads}, {num_key_value_heads}, {hidden_size}, {intermediate_size}, {head_dim}, {sum(p.numel() for p in model.parameters()) / 10**6:.1f}, {tloss:.4f}, {torch.Tensor(loss).mean():.4f}, {N_step}],')

2064.384
L24 att6 kv_heads3 hidden648 intermediate2592 head_dim272 T512


Step,Training Loss
144000,2.513500


[ 24, 6, 3, 648, 2592, 272, 229.8, 2.5135, 2.4748, 144000],


# L14 (Not used for figure because not explored enough)